In [178]:
import pandas as pd
import os

In [141]:
logsum = 'CFULL/SHO'

In [227]:
base_dir = r'U:\Stefan\soundcast_2014'
df_base = pd.read_csv(os.path.join(base_dir, r'outputs\aggregate_logsums.1.dat'), delim_whitespace=True, skipinitialspace=True)
df_base = df_base.reset_index()
df_base = pd.DataFrame(df_base[['level_0',logsum]])
df_base['source'] = '2014'

# Separate into accessibility bins
df_base['accessibility'] = pd.qcut(df_base[logsum],5,labels=['lowest','low','moderate','high','highest'])
bins = pd.qcut(df_base[logsum],5,retbins=True)[1]

df_base.columns = ['taz','logsum','source','accessibility']

# Attach population
hh = pd.read_csv(os.path.join(base_dir,r'outputs\_household.tsv'), sep='\t')
df_pop = pd.DataFrame(hh.groupby('hhtaz').sum()['hhsize'])
df_pop['taz'] = df_pop.index
df_base = pd.merge(df_base,df_pop,on='taz',how='left')
df_base.columns = [['taz','logsum','source','accessibility','population']]

In [228]:
scen_dir = r'S:\Stefan\soundcast_2040_plan'
df_scen = pd.read_csv(os.path.join(scen_dir, r'outputs\aggregate_logsums.1.dat'), delim_whitespace=True, skipinitialspace=True)
df_scen = df_scen.reset_index()
df_scen = pd.DataFrame(df_scen[['level_0','CFULL/SHO']])
df_scen['source'] = '2040 plan'

# Separate into accessibility bins
# ensure that bin start and end include any potential values for comparison
bins[0] = -999999999
bins[-1] = 999999999
df_scen['accessibility'] = pd.cut(x=df_scen[logsum],bins=bins,labels=['lowest','low','moderate','high','highest'])

df_scen.columns = ['taz','logsum','source','accessibility']

# Attach population
hh = pd.read_csv(os.path.join(scen_dir,r'outputs\_household.tsv'), sep='\t')
df_pop = pd.DataFrame(hh.groupby('hhtaz').sum()['hhsize'])
df_pop['taz'] = df_pop.index
df_scen = pd.merge(df_scen,df_pop,on='taz',how='left')
df_scen.columns = [['taz','logsum','source','accessibility','population']]

In [229]:
df = df_base.append(df_scen)

In [230]:
df.to_csv(r'c:/users/brice/logsums.txt', index=False)

In [231]:
# Save as 2 separate columns to compute differences
newdf = pd.merge(df[df['source'] == '2014'][['taz','logsum']],df[df['source'] == '2040 plan'][['taz','logsum']],on='taz', 
         suffixes=['_2014','2040_plan'])

In [232]:
# Calculate percent difference and separate into bins
newdf['% difference'] = (newdf['logsum2040_plan']-newdf['logsum_2014'])/newdf['logsum_2014']
newdf['% difference accessibility'] = pd.cut(x=newdf['% difference'],bins=5,labels=['lowest','low','moderate','high','highest'])

In [233]:
newdf.to_csv(r'c:/users/brice/logsums_compare.txt', index=False)

In [174]:
# taz sums
lu = pd.read_csv(r'R:\SoundCast\Inputs\2014\landuse\parcels_urbansim.txt', sep=' ')

In [175]:
lu.groupby('taz_p').sum().to_csv(r'J:\Projects\Soundcast\opportunity_index\2014_taz_landuse.csv')